## Advection equation using SBP Operators

Let us solve the one-dimensional advection equation with periodic boundary conditions
$$
u_t + c u_x = 0\;\; \text{on }[a,b], \\
c>0, \\
u(x,0)=u_0(x), \\
u(a,t)=u(b,t).
$$

Importing some libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from basis_nodes import generate_lagrange_poly, generate_LGL_points
from SBP_matrices import first_order_D, first_order_P_Q

Defining parameters

In [ ]:
#Computational domain
#Each element i will be enclosed by x_{i} and x_{i+1}
n_elements = 50
x = np.linspace(0,3*np.pi,n_elements+1)

#Advection speed
c = 1.
#Spatial order of accuracy
order_x = 4

Computing nodes at reference element

In [ ]:
#We'll need a Lagrange basis of degree order_x-1
#Thus, we will need order_x nodes in each element
degree_basis = order_x-1
(_, _,
_, _,
_, _,
xi_LGL_unsorted, w_LGL_unsorted) = generate_LGL_points(degree_basis)

#######################
#######################
#The root finder returns unsorted roots,
#We fix that in these lines

#Indexes that would sort x_LG using value
sort_idxs_LGL = np.argsort(xi_LGL_unsorted)
#Sorting x_LG and w_LG using those indexes
xi_LGL = xi_LGL_unsorted[sort_idxs_LGL]
w_LGL = w_LGL_unsorted[sort_idxs_LGL]
#######################
#######################

n_nodes = len(xi_LGL)

Mapping reference element to whole grid. The interface points are stored twice for simplicity.

In [ ]:
#We map the nodes from the reference element to one of the elements 
#from our computational domain (this suffices since we are using an uniform grid)
x_min = x[0]
x_max = x[1]

#We just need to map the LGL nodes once since we are using a uniform grid
x_element = (xi_LGL*(x_max-x_min)+(x_min+x_max))/2
w_element = (x_max-x_min)*w_LGL
list_elements = [x_element]

#We define our grid (interface points will be repeated)
x_grid = np.zeros(n_elements*n_nodes)
print(f"Numerical grid with double interface nodes")
for i in range(n_elements): 
    j = i*n_nodes
    x_grid[j:j+n_nodes] = x_element+x[i]
    list_elements.append(x_grid[j:j+n_nodes])
    print(f"Element {i+1:}\t {x_grid[j:j+n_nodes]}")


Setting initial conditions:
$
u(x,0) = \exp\left({-5(x-\pi)^2}\right)%\sin(x).
$

In [ ]:
#u = np.sin(x_grid)
initial_c = lambda x: np.exp(-5*(x-np.pi)**2)
u0 = initial_c(x_grid)
u = np.copy(u0)
n_grid = len(u)
plt.plot(x_grid,u0, label='$u(x,0)=\exp({-5(x-\pi)^2})$')
plt.legend()
plt.show()
plt.close()

Obtaining local relevant SBP operators and resizing them over the whole grid with Kronecker products

In [ ]:
#Local opretaros P and Q
P_LGL, Q_LGL = first_order_P_Q(x_Lagrange_nodes=x_element, x_abcissae=x_element, w_abcissae=w_element)

#Global operators P and Q
P = np.kron(np.eye(n_elements), P_LGL)
Q = np.kron(np.eye(n_elements), Q_LGL)

#Restriction operators R, B
#Local
R_LGL = np.zeros((2,n_nodes))
R_LGL[0,0] = R_LGL[-1,-1] = 1
B_LGL = np.zeros((2,2))
B_LGL[0,0] = 1; B_LGL[-1,-1]=-1
#Global
R = np.kron(np.eye(n_elements), R_LGL)
B = np.kron(np.eye(n_elements), B_LGL)


#Differential operator D
#Local
D_LGL = first_order_D(x_nodes=x_element)
#Global
D = np.kron(np.eye(n_elements), D_LGL)

#Numerical flux
#Local
#f_num_loc = lambda uL, uR : c*(uL+uR)/2 #Centered flux
f_num_loc = lambda uL, uR : c*uL #Upwind flux
#Global
#Global
def f_num(u,n_elements=n_elements,n_nodes=n_nodes):
    #Computing numerical flux just at the interface of elements
    #Fill numerical flux vector for first element
    #Using periodic BCs
    f = [c*u[-1],         #Left interface
         c*u[n_nodes-1]]  #Right interface
    for idx_elem in range(1,n_elements-1):
        #Left interface
        idx_R = idx_elem*n_nodes
        idx_L = idx_R-1
        f.append(f_num_loc(u[idx_L],u[idx_R]))
            #print(f"Element {idx_elem+1}")
            #print(f"Left interface")
            #print(f"{idx_L, idx_R}")
        #Right interface
        idx_R = (idx_elem+1)*n_nodes
        idx_L = idx_R-1
        f.append(f_num_loc(u[idx_L],u[idx_R]))
            #print(f"Right interface")
            #print(f"{idx_L, idx_R}")
        
    #Fill numerical flux vector for last element
    f.append(c*u[-n_nodes-1]) #Left interface
    f.append(c*u[-1])       #Right interface
        #print(f"Last element")
        #print(f"Left interface")
        #print(f"{-n_nodes-1, -n_nodes}")
        #print(f"Right interface")
        #print(f"{n_nodes*n_elements-1, 0}")
    return np.array(f)


#Vector and matrix with advection speed
c_vec = c*np.ones_like(u)
c_mat = c*np.eye(len(u))


def print_matrix(P):
    for i in range(len(P[0])):
        for j in range(len(P)):
            print(np.round(P[i,j],1),end=' ')
        print("")
    return None

Debugging cell:

In [ ]:
#We define our grid (interface points will be repeated)
x_grid = np.zeros(n_elements*n_nodes)
for i in range(n_elements): 
    j = i*n_nodes
    x_grid[j:j+n_nodes] = x_element+x[i]
    #print(f"Element {i+1:}\t {j}   {j+n_nodes-1}")

Debugging cell:

In [ ]:
f=f_num(u=u,n_elements=n_elements,n_nodes=n_nodes)
len(f)

Defining semi-discrete RHS operator

In [ ]:
def RHS(t,u):
    split_form_interior = -0.5*D@c_mat@u -0.5*c_mat@D@u -0.5*np.diag(u)@D@c_vec
    elem_boundary_terms = np.linalg.inv(P)@R.T@B@(f_num(u=u)-c*R@u)
    #print(np.max(f_num(u=u)-c*R@u))
    return split_form_interior+elem_boundary_terms

Plotting the solution

In [ ]:
from matplotlib.animation import FuncAnimation

def plotting_sol(u_frames,x_grid, t_eval, plotexact):
    # Set up the figure and axis
    fig, ax = plt.subplots(dpi=150, figsize=[6,3])
    ax.set_xlim(x_grid.min(), x_grid.max())
    ax.set_ylim(u_frames[0].min()-0.5, u_frames[0].max()+0.5)
    line_approx, = ax.plot(x_grid,u_frames[0])#ax.plot([], [])
    
    u_exact = np.array([initial_c(x_grid-c*time) for time in t_eval]) 
    if plotexact:
        line_exact, = ax.plot(x_grid,u_exact[0], linestyle='dotted', color='r')
    

    def animate(i):
        y_approx = u_frames[i]
        line_approx.set_data(x_grid, y_approx)
        if plotexact:
            line_exact.set_data(x_grid, u_exact[i])
            return line_approx, line_exact
        else:
            return line_approx,

    import matplotlib
    from matplotlib import animation
    from IPython.display import HTML
    anim = FuncAnimation(fig, animate,
                                       frames=len(u_frames),
                                       interval=200,
                                       repeat=False)
    plt.close()
    return HTML(anim.to_jshtml())
#plotting_sol(u_frames,x_grid)

## SciPy Time Integration

In [ ]:
from scipy.integrate import solve_ivp
t_span = [0,5.]
t_eval = np.linspace(t_span[0],t_span[1],20)
sol = solve_ivp(fun=RHS, y0=u0, t_eval=t_eval, t_span=t_span)
u_frames = sol.y.T
plotting_sol(u_frames, x_grid, t_eval, plotexact=True)

In [ ]:
#Computing numerical flux between each pair of nodes
#def f_num(u,n_elements=n_elements,n_nodes=n_nodes):
    #Fill numerical flux vector for first element
    #Using periodic BCs
    #f = []
    #for idx_node in range(len(u)):
        #Left interface of node idx_node
    #    L = idx_node-1; R = idx_node
    #    f.append(f_num_loc(u[L],u[R]))
    #return np.array(f)

### Custom Time Integration
Defining a time integrator for explicit  RK schemes

In [ ]:
def RK(u0, A, b, c, dt, tfinal):
    n = len(u0)
    t = 0.0
    u = u0
    u_frames = [u0]
    while t < tfinal:
        k = np.zeros((n, len(c)))
        for j in range(len(c)):
            uj = u + dt*np.dot(k, A[j])
            k[:, j] = RHS(t,uj)#, t + c[j]*dt)
        u = u + dt*np.dot(k, b)
        u_frames.append(u)
        t = t + dt
    return u_frames

Using Heun's method with Butcher tableau

\begin{array}{c|ccc}
0   & 0   & 0   & 0    \\
1/3 & 1/3 & 0   & 0    \\
2/3 & 0   & 2/3 & 0    \\
\hline
    & 1/4 & 0   & 3/4  \\
\end{array}

In [ ]:
A_Heun = np.array([[0., 0., 0.],
                   [1./3., 0., 0.],
                   [0., 2./3., 0.] ])
b_Heun = np.array([0.25, 0., 0.75])
c_Heun = np.array([0., 1./3., 2./3.])

Discretizing in time with custom RK integration

In [ ]:
#For nonlinear fluxes the time step should be computed at each time inside the time integrator
dx = np.min(np.diff(x_grid[:n_nodes]))
CFL = 0.5
dt = CFL*c*dx
#u_frames = RK(u0=u0, A=A_Heun, b=b_Heun, c=c_Heun, dt=dt, tfinal=10.)

In [ ]:
n_nodes = 4
u=np.ones(n_nodes); u[0]=1; u[-1]=2
#Restriction operators R, B
#Local
R_LGL = np.zeros((2,n_nodes))
R_LGL[0,0] = R_LGL[-1,-1] = 1
B_LGL = np.zeros((2,2))
B_LGL[0,0] = 1; B_LGL[-1,-1]=-1

In [ ]:
R_LGL@u
B_LGL@R_LGL@u
a=R_LGL.T@B_LGL@R_LGL@u
np.round(np.linalg.inv(P_LGL)@a,2)
#np.round(D_LGL,2)